In [ ]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES']='1'

solver = "ECOS"
save_data = False

from kelly_gambling import *
dataset = "kelly"

from osmm import OSMM
import pickle
import datetime

In [ ]:
max_num_rounds = 100

In [ ]:
alg_props = {    
    "r=0, M=1": [0, 0, 1],
    "r=0, M=20": [1, 0, 20],
    "r=0, M=50": [2, 0, 50],
    "r=20, M=1": [3, 20, 1],
    "r=20, M=20": [4, 20, 20],
    "r=20, M=50": [5, 20, 50],
    "r=50, M=1": [6, 50, 1],
    "r=50, M=20": [7, 50, 20],
    "r=50, M=50": [8, 50, 50],
}
    
num_algs = len(alg_props)

In [ ]:
# Define structs:
objfs = np.zeros((num_algs, max_num_rounds))
objfs_validation = np.zeros((num_algs, max_num_rounds))
runtimes = np.zeros((num_algs, max_num_rounds))
iters_takens = np.zeros(num_algs)
damping_facs = np.ones((num_algs, max_num_rounds))
mus = np.ones((num_algs, max_num_rounds))
ts = np.ones((num_algs, max_num_rounds))
Xs = np.zeros((n, num_algs, max_num_rounds))
lower_bounds = np.ones((num_algs, max_num_rounds)) * (-np.inf)
best_xs = np.zeros((n, num_algs))
v_norms = np.zeros((num_algs, max_num_rounds))
num_f_evals = np.zeros((num_algs, max_num_rounds))
rms_residuals = np.zeros((num_algs, max_num_rounds))
q_norms = np.zeros((num_algs, max_num_rounds))
f_grad_norms = np.zeros((num_algs, max_num_rounds))
time_cost_details = np.zeros((4, num_algs, max_num_rounds))
baseline_opt_objf = []
baseline_time_cost = []

In [ ]:
print("dataset = %s:" % (dataset))
W = generate_random_data()
W_validation = None
init = get_initial_val()
print("End generate data")

In [ ]:
osmm_prob = OSMM(my_f_torch, my_g_cvxpy)
osmm_prob.f_torch.W = W
osmm_prob.f_torch.W_validate = W_validation

In [ ]:
for alg_name in alg_props:
    print("    " + alg_name + ":")
    alg_idx = alg_props[alg_name][0]
    H_rank = alg_props[alg_name][1]
    M = alg_props[alg_name][2]
        
    osmm_prob.solve(init, max_iter=max_num_rounds, 
                    hessian_rank=H_rank, gradient_memory=M,
                    solver=solver, use_termination_criteria=False, verbose=True)

    Xs[:, alg_idx, :] = osmm_prob.method_results["var_iters"]
    objfs[alg_idx, :] = osmm_prob.method_results["objf_iters"]
    objfs_validation[alg_idx, :] = osmm_prob.method_results["objf_validate_iters"]
    lower_bounds[alg_idx, :] = osmm_prob.method_results["lower_bound_iters"]
    damping_facs[alg_idx, :] = osmm_prob.method_results["lam_iters"]
    mus[alg_idx, :] = osmm_prob.method_results["mu_iters"]
    ts[alg_idx, :] = osmm_prob.method_results["t_iters"]
    runtimes[alg_idx, :] = osmm_prob.method_results["time_iters"]
    best_xs[:, alg_idx] = osmm_prob.method_results["soln"]
    v_norms[alg_idx, :] = osmm_prob.method_results["v_norm_iters"]
    num_f_evals[alg_idx, :] = osmm_prob.method_results["num_f_evals_iters"]
    rms_residuals[alg_idx, :] = osmm_prob.method_results["rms_res_iters"]
    f_grad_norms[alg_idx, :] = osmm_prob.method_results["f_grad_norm_iters"]
    q_norms[alg_idx, :] = osmm_prob.method_results["q_norm_iters"]
    time_cost_details[:, alg_idx, :] = osmm_prob.method_results["time_detail_iters"]
    iters_takens[alg_idx] = osmm_prob.method_results["total_iters"]
    print("")

In [ ]:
print("W shape =", W.shape)
_, N = W.shape
print("log10 N =", np.log10(N), ", n =", n)

In [ ]:
stuff = {}
stuff["objfs"] = objfs
stuff["objfs_validation"] = objfs_validation
stuff["lower_bounds"] = lower_bounds
stuff["runtimes"] = runtimes
stuff["rms_residuals"] = rms_residuals
stuff["f_grad_norms"] = f_grad_norms
stuff["q_norms"] = q_norms
stuff["damping_facs"] = damping_facs
stuff["ts"] = ts
stuff["Xs"] = Xs
stuff["best_xs"] = best_xs
stuff["v_norms"] = v_norms
stuff["num_f_evals"] = num_f_evals
stuff["iters_takens"] = iters_takens
stuff["time_cost_details"] = time_cost_details
stuff["alg_props"] = list(alg_props.keys())

stuff["baseline_opt_objf"] = baseline_opt_objf
stuff["baseline_time_cost"] = baseline_time_cost
now = datetime.datetime.now()
mmddyyhhmm = ("%d_%d_%d_%d_%d" % (now.month, now.day, now.year, now.hour, now.minute))
part_of_out_fn = dataset + "" + mmddyyhhmm

In [ ]:
if save_data:
    pickle.dump(stuff, open("ECOS_data_%s.pkl" % (part_of_out_fn), "wb"))

In [ ]:
my_plot_one_result(W, best_xs[:, 0])

## timing results

In [ ]:
eps_gap_rel = 1e-3
eps_res_abs = 1e-4
eps_res_rel = 1e-3

In [ ]:
for alg_idx in range(len(alg_props)):
    stop_iters = [max_num_rounds] * 2
    for i in range(10, max_num_rounds):
        if objfs[alg_idx, i] < np.inf and \
        objfs[alg_idx, i] - lower_bounds[alg_idx, i] <= np.abs(objfs[alg_idx, i] - objfs_validation[alg_idx, i]) + eps_gap_rel * np.abs(objfs[alg_idx, i]):
            stop_iters[0] = min(i, stop_iters[0])
        if ts[alg_idx, i] == 1 and \
        rms_residuals[alg_idx, i] <= eps_res_abs + eps_res_rel * (q_norms[alg_idx, i] + f_grad_norms[alg_idx, i]) / np.sqrt(n):
            stop_iters[1] = min(i, stop_iters[1])
        if max(stop_iters) < max_num_rounds:
            break
    print("iteration at which a stopping condition holds", stop_iters)
    stop_iter = min(stop_iters)
    solve_time = np.sum(runtimes[alg_idx, 0:np.min(stop_iters) + 1])
    f_evals = np.mean(num_f_evals[alg_idx, 1:np.min(stop_iters) + 1], axis=0)
    print("alg_idx =", alg_idx, ", solve time (stopping criteria) = ", '{:g}'.format(float('{:.2g}'.format(solve_time))),
    ", iterations=", np.min(stop_iters), ", f evals=", '{:g}'.format(float('{:.2g}'.format(f_evals))))

In [ ]:
ep = 1e-6
for alg_idx in range(len(alg_props)):
    subopt = objfs[alg_idx, :] - np.min(objfs[alg_idx, :])
    iter_reach_ep = max_num_rounds
    for i in range(max_num_rounds):
        if subopt[i] <= ep:
            iter_reach_ep = i
            break
    f_evals = np.mean(num_f_evals[alg_idx, 1:iter_reach_ep + 1], axis=0)
    print("iters to reach 1e-6 subopt = ", iter_reach_ep)
    print("solve time to reach 1e-6 subopt = ", np.sum(runtimes[alg_idx, 0:iter_reach_ep + 1]))
    print("f evals", f_evals)

In [ ]:
# mean_time_details = np.mean(time_cost_details, axis=3)
print("time cost detail")
print("===============f======grad f====subp====L_k====")
for alg_idx in range(len(alg_props)):
    f_eval = np.mean(time_cost_details[0, alg_idx, 1::])
    f_grad_eval = np.mean(time_cost_details[1, alg_idx, 1::])
    subp_ave = np.mean(time_cost_details[2, alg_idx, 2::])
    subp_first = time_cost_details[2, alg_idx, 1]
    L_k_ave = np.mean(time_cost_details[3, alg_idx, 20:100:10])
    L_k_first = time_cost_details[3, alg_idx, 10]
    print_results = [f_eval, f_grad_eval, subp_ave, L_k_ave]
    print(alg_idx, ['{:g}'.format(float('{:.2g}'.format(print_results[i]))) for i in range(4)])